## 实验题目5 高斯(Gauss)列主元消去法

### 代码实现

In [63]:
using Printf
using LinearAlgebra

In [64]:
# from: https://stackoverflow.com/questions/58667332/is-there-a-way-to-swap-columns-in-o1-in-julia
function swapcols!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X,1)
        X[k,i], X[k,j] = X[k,j], X[k,i]
    end
end
# from: https://discourse.julialang.org/t/swap-cols-rows-of-a-matrix/47904/9
function _swapcol!(x,i,j)
    for k in axes(x, 1)  # <- give dimension as input to axes function
        x[k, i], x[k, j] = x[k, j], x[k, i]
    end
end

_swapcol! (generic function with 1 method)

In [65]:
    function swaprows!(X::AbstractMatrix, i::Integer, j::Integer)
        @inbounds for k = 1:size(X,2)
            X[i,k], X[j,k] = X[j,k], X[i,k]
        end
    end

swaprows! (generic function with 1 method)

In [66]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64})
    for k = 1:n-1
        val, idx = findmax(A[k:n, k])
        idx += k - 1  # index must add previous length that omitted by slice operator
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 1 method)

In [67]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss_explicit(n, A::Matrix{Float64}, b::Vector{Float64})
    for k = 1:n-1
        s = [maximum(A[i,k:n]) for i in k:n]
        # display(s)
        if 0 in s
            println("Cannot solve a singular matrix!")
            return
        end
        A[k:n,k:n] = A[k:n,k:n] ./ s
        b[k:n] = b[k:n] ./ s
        
        val, idx = findmax(A[k:n, k])
        idx += k - 1  # index must add previous length that omitted by slice operator
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss_explicit (generic function with 1 method)

In [68]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss_implicit(n, A::Matrix{Float64}, b::Vector{Float64})
    for k = 1:n-1
        s = [maximum(A[i,k:n]) for i in k:n]
        # display(s)
        if 0 in s
            println("Cannot solve a singular matrix!")
            return
        end
        # A[k:n,k:n] = A[k:n,k:n] ./ s
        # b[k:n] = b[k:n] ./ s
        
        val, idx = findmax(A[k:n, k] ./s[1:n-k+1])
        idx += k - 1  # index must add previous length that omitted by slice operator
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss_implicit (generic function with 1 method)

### Test 1

In [69]:
# test random result of standard library 
# test pass
for i in 1:100
    A = rand(4, 4)
    b = rand(4)
    display(norm(A \ b - gauss(size(A, 1), A, b),2))
    # display(norm(A \ b - gauss_explicit(size(A, 1), A, b),2))
    # display(norm(A \ b - gauss_implicit(size(A, 1), A, b),2))
    # display([A \ b  gauss_explicit(size(A, 1), A, b)])
    
end

# A = [0.1 0.1; 2.3 2.3]
# b = [1.0, 3]
# try
#     A \ b
# catch SigularException
#     println("Cannot solve a singular matrix!")
# end

5.4672143489065705e-16

5.551115123125783e-17

1.631687946612622e-15

1.6653345369377348e-16

9.945640348601413e-16

...

1.249000902703301e-16

1.3732700395566711e-15

5.48304353729612e-16

2.220446049250313e-16

1.9860273225978185e-15